# Diversity Visa Lottery Analysis

By [Wikipedia](https://en.wikipedia.org/wiki/Diversity_Immigrant_Visa) definition "Diversity Immigrant Visa is a United States government lottery program for receiving a United States Permanent Resident Card."

The first lottery was in 1995 and since then it has been done annually. Even though the visa limit is 55.000 in total, the number of applicants has been increasing every year which means there is less and less chance of winning every following year in some countries where number of applicants increases significantly. 

In this project I wanted to see in which countries applicants have better or worse chance of winning and visualize it as an interactive map.

Data was acquired from [travel.state.gov](https://travel.state.gov/content/travel/en/us-visas/immigrate/diversity-visa-program-entry/diversity-visa-program-statistics.html).

### Table of contents
- [Reading the Data](#Reading-the-Data)
- [Preparing the Data](#Preparing-the-Data)
- [Visualization with Tableau](#Visualization-with-Tableau)
- [Conclusion](#Conclusion)

## Reading the Data

I'll start with importing pandas library and reading the data into pandas dataframe.

In [1]:
# Import pandas
import pandas as pd

The tables for number of entries are structured for 3 year time period at a time and they show the number of primary entrants, their derivatives(family etc.) and total entrants. Since I will use only the number of total entrants for each year, I will extract column 3,6 and 9 from each dataframe and drop the other columns that I will not use. You can see a screenshot of what data looks like in the original source below.

![alt text](DV_table.png "DV_table")



In [2]:
# Read the csv data into a dataframe
entrants_10_12 = pd.read_csv('data/number_of_entries_2010_2012.csv', header = None)
entrants_10_12.head()

,0,1,2,3,4,5,6,7,8,9
0,Afghanistan,"39,446","27,424","66,870","9,627","10,029","19,656","5,519","6,937","12,456"
1,Albania,"53,104","65,809","118,913","40,823","46,366","87,189","55,686","59,671","115,357"
2,Algeria,"68,052","22,652","90,704","66,404","21,600","88,004","85,568","26,513","112,081"
3,Andorra,276,55,331,79,26,105,129,39,168
4,Angola,883,934,"1,817","1,039","1,291","2,330","1,391","1,664","3,055"


In [3]:
# Read the csv data into pandas dataframe
entrants_10_12 = pd.read_csv('data/number_of_entries_2010_2012.csv', header = None)

# Drop the columns that we don't need
entrants_10_12 = entrants_10_12.drop(entrants_10_12.columns[[1,2,4,5,7,8]], axis =1)

# Change the column names with years
entrants_10_12.columns = ['Country', '2010', '2011', '2012']

entrants_13_15 = pd.read_csv('data/number_of_entries_2013_2015.csv' , header = None)
entrants_13_15 = entrants_13_15.drop(entrants_13_15.columns[[1,2,4,5,7,8]], axis =1)
entrants_13_15.columns = ['Country', '2013', '2014', '2015']

entrants_16_18 = pd.read_csv('data/number_of_entries_2016_2018.csv' , header = None)
entrants_16_18 = entrants_16_18.drop(entrants_16_18.columns[[1,2,4,5,7,8]], axis =1)
entrants_16_18.columns = ['Country', '2016', '2017', '2018']

In [4]:
# Combine all the tables to create one from 2010 to 2018
combined_1 = entrants_10_12.merge(entrants_13_15, how = 'left', on = 'Country')
combined = combined_1.merge(entrants_16_18, how = 'left', on = 'Country')
combined.head()

,Country,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Afghanistan,"66,870","19,656","12,456","16,277","31,485","27,789","65,951","83,277","122,755"
1,Albania,"118,913","87,189","115,357","122,567","171,362","198,625","246,129","315,842","367,231"
2,Algeria,"90,704","88,004","112,081","128,189","127,872","163,091","204,433","252,733","342,857"
3,Andorra,331,105,168,147,144,134,186,166,175
4,Angola,"1,817","2,330","3,055","3,578","5,884","9,198","11,546","14,859","22,819"


## Preparing the Data

In order to be able to do statistical analysis I will clean the data and make all values integers. First, let's check the attribute types.

In [5]:
combined.dtypes

Country    object
2010       object
2011       object
2012       object
2013       object
2014       object
2015       object
2016       object
2017       object
2018       object
dtype: object

In [6]:
# Normalize text
import re

# Replace non-word characters with empty space
def normalize_text(text):
    text = str(text)
    text = re.sub('[^A-Za-z0-9\s]', ' ', text)
    return text

# Text to title format
def title_text(text):
    text = text.title()
    text = text.rstrip()
    return text

# Text to integer
def normalize_value(text):
    text = re.sub('\s', '', text)
    try:
        value = int(text)
    except:
        value = 0
    return value

In [7]:
# Apply the functions to the dataframe
combined.iloc[:, 1:] = combined.iloc[:, 1:].applymap(normalize_text)
combined.iloc[:, 1:] = combined.iloc[:, 1:].applymap(normalize_value)

# Apply the function to 'Country' column
combined.iloc[:, 0] = combined.iloc[:, 0].apply(title_text)

### Statistical Calculations
In this step, I want to find how number of applications increase/decrease in over the course of 8 years for every country in terms of percentage. And also I want to see what the average number of applications and the chance of winning for applicants in each country.

### Part-1

In [8]:
# Calculate number of applications difference between 2010 and 2018
combined['8_years_diff'] = combined['2018'] - combined['2010']

# Calculate the percentage of increase/decrease from 2010 to 2018 in the number of applications 
combined['%_increase'] = (combined[combined['2018'] > 0]['2018'] / combined[combined['2010'] > 0]['2010'] *100) - 100

# Replace null values with 0
combined = combined.fillna(0)

# Calculate the average number of applications for every country
combined['average'] = round(combined.iloc[:,1:10].mean(axis = 1))
combined = combined.sort_values(['average'], ascending = False)

# Rename the dataframe
DV_num_of_entries = combined


In [9]:
# Save the dataframe as a csv file
DV_num_of_entries.to_csv('data/DV_num_of_entries.csv')

In [10]:
DV_num_of_entries.iloc[1:,:].sort_values('average', ascending = False)

,Country,2010,2011,2012,2013,2014,2015,2016,2017,2018,8_years_diff,%_increase,average
14,Bangladesh,834324,6497926,8562251,0,0,0,0,0,0,-834324,0.000000,1766056.0
69,Ghana,523448,609924,774557,1056032,1217888,1729979,2231745,2199021,2227530,1704082,325.549434,1396680.0
198,Ukraine,927470,1080091,1122086,1232306,1255129,1274758,1291999,1470250,1450487,523017,56.391797,1233842.0
134,Nigeria,1570316,2144626,2005876,1975571,2390758,0,0,0,0,-1570316,0.000000,1120794.0
201,Uzbekistan,194033,295828,507361,654327,1193657,1387420,1488984,1576179,2114446,1920413,989.735251,1045804.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,Saint Barthelemy,0,0,0,58,82,18,0,0,0,0,0.000000,18.0
36,Christmas Island,18,5,13,6,19,17,12,16,15,-3,-16.666667,13.0
117,Mayotte,0,0,0,16,36,23,18,0,0,0,0.000000,10.0
37,Cocos Islands,5,8,10,35,25,11,0,0,0,-5,0.000000,10.0


It seems like when we sort the countries by their average number of applications, first 5 countries have average of more than 1 million lottery application. And it seems like the U.S. stopped taking application from Bangladesh and Nigeria at some point. It may be because of the country quota that the U.S. had set. We can analyze it as number of applications to population ratio in further steps. If we look at the most bottom row, we see that the U.S. did not take any application from Brazil between the years of 2010 and 2018. 

Let's sort the top 15 countries with the largest average number of applications by their increase rates.

In [11]:
DV_num_of_entries.iloc[1:,:].sort_values('average', ascending = False).head(15).sort_values('%_increase', ascending = False).head()

,Country,2010,2011,2012,2013,2014,2015,2016,2017,2018,8_years_diff,%_increase,average
201,Uzbekistan,194033,295828,507361,654327,1193657,1387420,1488984,1576179,2114446,1920413,989.735251,1045804.0
39,"Congo, Democratic Republic Of The",92133,138558,208775,229650,339196,12581,500395,615454,775293,683160,741.493276,323559.0
100,Liberia,119229,110960,141749,142298,216249,358224,661631,674936,686869,567640,476.092226,345794.0
69,Ghana,523448,609924,774557,1056032,1217888,1729979,2231745,2199021,2227530,1704082,325.549434,1396680.0
83,Iran,470279,426505,547735,692410,1037354,932346,1203531,1390853,1624204,1153925,245.370301,925024.0


It seems like all top 15 countries are developing countries. The most significant increase in number of applications has happened in Uzbekistan from 2010 to 2018 with almost 1000%.

### Part-2
We'll continue with analyzing number of selected entrants between the years of 2016 and 2018 and calculate the odds of winning. I'll repeat the cleaning in this part for the new dataframes I'll be using too.

In [15]:
# Read the csv files into pandas dataframes
DV_selected_entrants_16 = pd.read_csv('data/DV_selected_entrants_2016.csv', header = None)
DV_selected_entrants_17 = pd.read_csv('data/DV_selected_entrants_2017.csv', header = None)
DV_selected_entrants_18 = pd.read_csv('data/DV_selected_entrants_2018.csv', header = None)

In [16]:
DV_selected_entrants_16.head()

,0,1
0,"ALGERIA 1,952",NaN
1,"ETHIOPIA 4,000",NaN
2,NIGER 102,NaN
3,ANGOLA 108,NaN
4,GABON 32,NaN


In [17]:
# Rename the columns
DV_selected_entrants_16.columns = ['Country', '2016']
DV_selected_entrants_17.columns = ['Country', '2017']
DV_selected_entrants_18.columns = ['Country', '2018']

In [18]:
# Separate numbers from country names and create another column for number of selected entrants
def clean_table(df):
    df.iloc[:, 1] = df.iloc[:, 0].str.extract(r'(\d.*)') # Second column for capturing the part 
                                                         # starting with a digit and followed by any character
    df.iloc[:, 0] = df.iloc[:, 0].str.extract(r'(\D*\s)')# First column for the part starting 
                                                         # with non-digit character and followed by a white space
    
    # after using encode('utf-8') it appears that the Country names are not consistent on the tables
    df.iloc[:, 0] = df.iloc[:, 0].str.replace('\xa0', ' ')
    return df
    

In [19]:
# Apply the function to all dataframes
clean_table(DV_selected_entrants_16)
clean_table(DV_selected_entrants_17)
clean_table(DV_selected_entrants_18)

# Merge dataframes on country names
pd.set_option('display.max_rows', 200)
DV_17_18 = DV_selected_entrants_17.merge(DV_selected_entrants_18, how = 'right', on = 'Country')
DV_selected_entrants = DV_selected_entrants_16.merge(DV_17_18, how = 'right', on = 'Country')

# Fill null values with 0
DV_selected_entrants = DV_selected_entrants.fillna(0)
DV_selected_entrants.head()

,Country,2016,2017,2018
0,ALGERIA,"1,952","1,561","2,897"
1,ETHIOPIA,"4,000","4,500","4,496"
2,NIGER,102,34,57
3,ANGOLA,108,80,184
4,GABON,32,26,41


In [20]:
def title_text(text):
    text = text.title() # Country names title formatted
    text = text.rstrip() # Remove trailing characters
    return text

In [21]:
# Apply the function to 'Country' column
DV_selected_entrants['Country'] = DV_selected_entrants['Country'].apply(title_text)

# Save the dataframe as a csv file
DV_selected_entrants.to_csv('data/DV_selected_entrants.csv')

# Sort values alphabetically by country names
DV_selected_entrants = DV_selected_entrants.sort_values('Country', ascending = True)
DV_selected_entrants.head()

,Country,2016,2017,2018
53,Afghanistan,406,285,636
85,Albania,"1,931","2,373","4,484"
0,Algeria,"1,952","1,561","2,897"
88,Andorra,0,0,0
3,Angola,108,80,184


In [22]:
# Rename the columns
DV_selected_entrants.columns = ['Country', 's2016', 's2017', 's2018']

After almost cleaning the data for the number of selected entrants, I will merge it with our previus dataframe that shows the number of applicants to calculate the odds of winning for the years of 2016, 2017 and 2018.

In [23]:
# We only want to select the years of 2016, 2017 and 2018 to merge 
combined = combined.iloc[:, [0,7,8,9]].merge(DV_selected_entrants, how = 'left', on = 'Country')
with pd.option_context("display.max_rows", 1000):
    display(combined.head())


,Country,2016,2017,2018,s2016,s2017,s2018
0,Grand Total,17573364,19344586,23088613,NaN,NaN,NaN
1,Bangladesh,0,0,0,NaN,NaN,NaN
2,Ghana,2231745,2199021,2227530,"3,179","3,170","3,549"
3,Ukraine,1291999,1470250,1450487,"4,507","4,500","4,478"
4,Nigeria,0,0,0,NaN,NaN,NaN


NaN values mean there is no selected entrant from those countries at given years. So we will replace them with 0. And we will unify all integers because of the inconsistency between combined tables.

In [24]:
#change all NaN values as 0
combined = combined.fillna(0)
combined.head()

,Country,2016,2017,2018,s2016,s2017,s2018
0,Grand Total,17573364,19344586,23088613,0,0,0
1,Bangladesh,0,0,0,0,0,0
2,Ghana,2231745,2199021,2227530,"3,179","3,170","3,549"
3,Ukraine,1291999,1470250,1450487,"4,507","4,500","4,478"
4,Nigeria,0,0,0,0,0,0


In [25]:
def unify_value(text):
    if type(text) == str:
        text = re.sub('\D', '', text) # Remove non-digit characters
        value = int(text) # Turn text to integer
    else:
        value = text
    return value


# Apply the function to the dataframe except for Country column
combined.iloc[:, 1:] = combined.iloc[:, 1:].applymap(unify_value)
combined.head()

,Country,2016,2017,2018,s2016,s2017,s2018
0,Grand Total,17573364,19344586,23088613,0,0,0
1,Bangladesh,0,0,0,0,0,0
2,Ghana,2231745,2199021,2227530,3179,3170,3549
3,Ukraine,1291999,1470250,1450487,4507,4500,4478
4,Nigeria,0,0,0,0,0,0


In [26]:
# Avoid SettingWithCopyWarning
pd.set_option('mode.chained_assignment', None)

# Calculate Grand Total for number of selected entrants
combined['s2016'][0] = sum(combined['s2016'][1:])
combined['s2017'][0] = sum(combined['s2017'][1:])
combined['s2018'][0] = sum(combined['s2018'][1:])

In [27]:
combined.head()

,Country,2016,2017,2018,s2016,s2017,s2018
0,Grand Total,17573364,19344586,23088613,87108,83904,115957
1,Bangladesh,0,0,0,0,0,0
2,Ghana,2231745,2199021,2227530,3179,3170,3549
3,Ukraine,1291999,1470250,1450487,4507,4500,4478
4,Nigeria,0,0,0,0,0,0


In [28]:
# Create new dataframe for the winning chances for each country between 2016-2018
data = {'Country': combined['Country'],
        '2016': (combined['s2016']/combined['2016']*100),
        '2017': (combined['s2017']/combined['2017']*100),
        '2018': (combined['s2018']/combined['2018']*100)}

winning_chances = pd.DataFrame(data)
winning_chances = winning_chances.fillna(0)
with pd.option_context("display.max_rows", 1000):
    display(winning_chances.sort_values('Country', ascending = True))


,Country,2016,2017,2018
43,Afghanistan,0.615609,0.342231,0.518105
18,Albania,0.784548,0.751325,1.221030
20,Algeria,0.954836,0.617648,0.844959
176,Andorra,0.000000,0.000000,0.000000
97,Angola,0.935389,0.538394,0.806346
153,Antigua And Barbuda,0.000000,1.716069,0.380952
76,Argentina,0.383596,0.289475,0.550199
25,Armenia,0.759599,0.736719,1.064901
152,Aruba,0.663350,0.335570,1.309329
69,Australia,3.342977,2.801078,8.173642


In [29]:
# Save the dataframe as a csv file
winning_chances.to_csv('data/winning_chances.csv')

## Visualization with Tableau
In this step, I will create an interactive map with Tableau where you can zoom in/out and see the average number of applicants per year and total number of applicants between 2010 and 2018 in each country with their country name. 

Also, you will be able to see the change in number of applications for any selected country for over the course of 8 years and the chances of winning the lottery for the given country between 2016 and 2018. 

In [1]:
%%html
<div class='tableauPlaceholder' id='viz1600030767149' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;GT&#47;GTBS4ZNFD&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='path' value='shared&#47;GTBS4ZNFD' /> <param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;GT&#47;GTBS4ZNFD&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1600030767149');                    var vizElement = divElement.getElementsByTagName('object')[0];                    if ( divElement.offsetWidth > 800 ) { vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';} else if ( divElement.offsetWidth > 500 ) { vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';} else { vizElement.style.width='100%';vizElement.style.height='727px';}                     var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

## Conclusion
In conclusion, while people in some countries have better chance to win the lottery due to the country quota that the U.S. set and the number of applicants, people in other countries have less chance because of the increasing number of applications every year.